<a href="https://colab.research.google.com/github/kevindinh24/BERT-Fine-Tuning-on-GLUE/blob/main/BERT_on_qqp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install torch

In [ ]:
from datasets import load_dataset
dataset = load_dataset("glue", "qqp")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


qqp/train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

qqp/validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

qqp/test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer #Loads pre trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["question1"], examples["question2"], truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #Pads input to be same length to save memory

In [ ]:
%env WANDB_DISABLED=true

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#training arguemnts
training_args = TrainingArguments("test_trainer")

#Create trainer instance
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer
)

trainer.train()

env: WANDB_DISABLED=true


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-132911641.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.523100
1000,0.503100
1500,0.485500
2000,0.462200
2500,0.450800
3000,0.450200
3500,0.459200
4000,0.434200
4500,0.435100
5000,0.436000


SafetensorError: Error while serializing: I/O error: No space left on device (os error 28)

In [ ]:
!pip install evaluate

import numpy as np
import evaluate

predictions = trainer.predict(tokenized_datasets["validation"])

pred_labels = np.argmax(predictions.predictions, axis=1)

true_labels = predictions.label_ids

metric = evaluate.load("glue", "qqp")

result = metric.compute(predictions=pred_labels, references=true_labels)

print(result)

Step,Training Loss
500,0.523100
1000,0.503100
1500,0.485500
2000,0.462200
2500,0.450800
3000,0.450200
3500,0.459200
4000,0.434200
4500,0.435100
5000,0.436000


{'accuracy': 0.6322532772693544, 'f1': 0.0024154589371980675}


In [ ]:
!rm -rf ~/.cache/huggingface